In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn


In [2]:
cars_file = 'https://gist.githubusercontent.com/noamross/e5d3e859aa0c794be10b/raw/b999fb4425b54c63cab088c0ce2c0d6ce961a563/cars.csv'
cars = pd.read_csv(cars_file)

sns.scatterplot(x='wt', y='mpg', data=cars)
sns.regplot(x='wt', y='mpg', data=cars)


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
cars.head()


In [ ]:
x_list = cars.wt.values
x_list = np.array(x_list, dtype=np.float32).reshape(-1, 1)
x_list


In [ ]:
y_list = cars.mpg.values
y_list = np.array(y_list, dtype=np.float32)
y_list


In [ ]:
X = torch.from_numpy(x_list)
Y = torch.tensor(y_list)
X, Y


In [ ]:
w = torch.rand(1, requires_grad=True, dtype=torch.float32)
b = torch.rand(1, requires_grad=True, dtype=torch.float32)

num_epochs = 1000
lr = 1e-03

for epoch in range(num_epochs):
    for i in range(len(X)):
        # forward pass
        y_pred = w*X[i] + b

        # loss calc
        loss = (y_pred-Y[i])**2

        # Gradient Calc
        loss.backward()

        # extract losses
        loss_val = loss.data[0]

        # update weights and biases
        with torch.no_grad():
            w -= w.grad*lr
            b -= b.grad*lr
            w.grad.zero_()
            b.grad.zero_()

    if ((epoch+1) % 100 == 0):
        print(f'epoch: {epoch+1} , loss: {loss_val}')


In [ ]:
y_list


In [ ]:
print(f'weights: {w.data}, bias: {b.data}')

y_pred = (w*X + b).detach().numpy()
print(y_pred.reshape(-1))
sns.scatterplot(x=x_list.reshape(-1), y=y_list)
sns.lineplot(x=x_list.reshape(-1), y=y_pred.reshape(-1))


In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_list, y_list)
print(f'slope: {reg.coef_} intercept: {reg.intercept_}')


In [ ]:
# from torchviz import make_dot
# import os
# os.environ['PATH'] += os.pathsep+"C:/Program Files (x86)/Graphviz/bin"
# make_dot(loss)


## Model CLass

In [ ]:
class LinearRegressionTorch(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        pred = self.linear(x)
        return pred


inputd = outputd = 1
model = LinearRegressionTorch(inputd, outputd)


In [ ]:
loss_func = nn.MSELoss()
lr = 0.002
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

losses, slope, bias = [], [], []

X = torch.from_numpy(x_list.astype(np.float32)).reshape(-1,1)
Y_true = torch.from_numpy(y_list.astype(np.float32)).reshape(-1,1)

epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()  # Set gradients to 0

    # forward pass
    y_pred = model(X)

    # compute loss
    loss = loss_func(y_pred, Y_true)

    # update weights
    optimizer.step()

    # get params
    for name, param in model.named_parameters():
        if param.requires_grad:
            if name == "linear.weight":
                slope.append(param.data.numpy()[0][0])
            if name == "linear.bias":
                bias.append(param.data.numpy()[0])

    # store loss
    losses.append(float(loss.data))

    if ((epoch+1) % 100 == 0):
        print(f"Epoch {epoch}, Loss: {loss.data}")

In [ ]:
# Visualize loss over each epoch
sns.scatterplot(x=range(epochs), y=losses)

In [ ]:
# Visualize the bias development and slope development
sns.lineplot(x=range(epochs), y=bias)
sns.lineplot(x=range(epochs), y=slope)

In [ ]:
# check results
y_pred = model(torch.tensor(x_list)).data.numpy().reshape(-1)
sns.scatterplot(x=x_list.reshape(-1), y=y_list)
sns.lineplot(x=x_list.reshape(-1), y=y_pred,color="red")

